### Clustering and filtering viral sequences:
- We have low, high and medium quality sequences. Since these come from metaGs, I only want to keep high and med quality. 
- I have done this before, but the results are a mess so will redo
- Need to include all new vOTUs
- Then cluster using CD-HIT: https://www.biostars.org/p/366171/ for full file names


only use the 339 'good' datasets for now. 
Cluster these at 95% ANI, and then look for clusters that we can investigate for pangenomics. 

In [ ]:
# concatenate all viral score tsvs (n=111,082)
mamba activate csvtk
csvtk concat virsorter2/*/final-viral-score.tsv -t > virsorter2_viralscores.tsv

In [ ]:
# Rename the csv files with viral scores
for f in */final-viral-combined.fa
do 
echo $f
grep -e '>' $f > ../header_files/oldhead/${f%/*}.csv
done

# now for the renamed ones 
for f in *rename*.fa
do
echo $f
grep -e '>' $f > ../header_files/newhead/${f%_rename*}.csv
done

# remove the carrots
for f in *.csv
do
sed 's/[<>,]//g' $f > ${f%.csv*}.clean.csv
done

In [ ]:
# now make a concatenated file with names
for f in *.clean.csv
do
paste -d "\t" $f ../newhead/$f > ../combined_head/$f
done

# add header
for f in *.csv
do
csvtk add-header -t $f -n seqname,newname > $f.newname
done

# concat all
csvtk concat -t *.newname > ../header_keys.tsv

In [ ]:
# join the final csvs
csvtk join -t -f seqname virsorter2_viralscores.tsv header_files/header_keys.tsv > renamed_viralscore.tsv

# select for max_score => 0.9 (n=62,146)
csvtk filter -f "max_score>=0.9" -t renamed_viralscore.tsv > highqual_renamed_viralscore.tsv
csvtk filter -f "length>=5000" -t highqual_renamed_viralscore.tsv > highqual_renamed_viralscore.len.tsv


# take the column with the new names so we can filter the fasta
csvtk cut -f "newname" highqual_renamed_viralscore.len.tsv -t > highqual_namelist.txt

In [ ]:
# now use bbmap to filter fasta
mamba activate bbmap 
filterbyname.sh in=hq_virseqs.fa out=hq_virseqs.len.fa names=./tsv_files/highqual_namelist.txt include=t

# and sort it by length 
sortbyname.sh in=hq_virseqs.len.fa out=hq_virseqs.sort.fa length descending

In [ ]:
# now cdhit to deduplicate 
srun --account=ctbrowngrp -p bmm -J cdhit -t 5:00:00 -c 32 --mem=70gb --pty bash

mamba activate cdhit
cd-hit-est -i hq_virseqs.sort.fa \
-o hq_virseqs.95.cluster.fa -d 0 \
-c 0.95 -aS 0.85 -M 70000 -T 32